# APRENDENDO SOBRE TTQ - TEXT TO QUERY  

Frameworks de agentes são fascinantes! Eles permitem a execução de uma série de tarefas que antes eram extremamente complicadas - ou até mesmo impossíveis.  
Trabalho criando consultas em bancos de dados desde 2018 e, quando os LLMs foram lançados, logo me perguntei:  

> Será que é possível pedir para uma LLM gerar uma consulta SQL e executá-la? 🤔  

Bem, é exatamente isso que vou testar neste notebook.  

## **Motivação**  

Uma das minhas principais atividades é digitalizar e automatizar processos de negócio. Dentro desse contexto, estabeleci um desafio:  

> Como criar um sistema que interprete uma solicitação em linguagem natural e gere uma consulta SQL sobre um processo?  

Acredito que esse objetivo é importante, pois, criar um sistema que transforma perguntas em **consultas SQL válidas** pode ser extremamente útil, uma vez que permitem:  

✅ **Melhorar a usabilidade** de sistemas, permitindo interações mais naturais com bases de dados. 

## Importando bibliotecas

In [1]:
from typing import List, Any, Dict
import re
import os
import sqlite3
import logging
import time
import json
from datetime import datetime
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_groq import ChatGroq
from dotenv import load_dotenv

## Carregando Variáveis de Ambiente

In [2]:
load_dotenv()

DB_PATH = "../.db/SQL_AGENT.db"

## Criando logger

In [3]:
logging.basicConfig(   
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[        
        logging.StreamHandler()
    ]
)

LOGGER = logging.getLogger(__name__)
LOGGER.setLevel(logging.INFO)

### 🏗️ Criando o Banco de Dados  

Antes de qualquer coisa, precisamos de um **banco** com uma **tabela** e algumas informações para pesquisar. 📊🔍  

Imagine uma empresa que gerencia processos como **Recrutamento**, **Seleção**, **Avaliação de Desempenho** e **Solicitação de Férias**, todos mapeados e digitalizados dentro da plataforma **Lecom**. 🏢✅  

Depois, foi criado um processo que **extrai informações importantes** sobre esses fluxos e as armazena na tabela **`processos_andamento`**. 📂 Essa será a **base de dados** utilizada pelo nosso sistema de agentes. 🤖⚙️  

Vale destacar que tudo isso foi criado de forma **genérica**, com a ajuda do **ChatGPT**. 🚀

In [4]:
LOGGER.info(f"Iniciando configuração do banco de dados em {DB_PATH}")

os.makedirs(os.path.dirname(DB_PATH), exist_ok=True)

try:
    conn = sqlite3.connect(DB_PATH)
    cursor = conn.cursor()
    LOGGER.info("Conexão com o banco de dados estabelecida com sucesso")
   
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table' AND name='processos_andamento'")
    if cursor.fetchone():       
        cursor.execute("DROP TABLE processos_andamento")
   
    LOGGER.info("Criando tabela 'processos_andamento'")
    cursor.execute("""
    CREATE TABLE processos_andamento (
        ID_Processo_Andamento INTEGER PRIMARY KEY AUTOINCREMENT,
        Codigo_Processo INTEGER NOT NULL,
        Codigo_Atividade INTEGER NOT NULL,
        Nome_Processo TEXT NOT NULL,
        Nome_Atividade TEXT NOT NULL,
        Nome_Cliente TEXT NOT NULL,
        Telefone_Cliente TEXT NOT NULL,
        Descricao_Processo TEXT NOT NULL,
        Data_Atividade DATE NOT NULL
    );
    """)
   
    dados_iniciais = [
        (1, 101, 'Recrutamento', 'Receber currículo', 'João Silva', '11987654321', 'Recebeu currículo e iniciou análise.', '2024-03-01'),
        (1, 102, 'Recrutamento', 'Entrevista inicial', 'João Silva', '11987654321', 'Entrevista marcada para avaliação inicial.', '2024-03-02'),
        (2, 201, 'Seleção', 'Teste técnico', 'Maria Oliveira', '11976543210', 'Teste técnico agendado.', '2024-03-03'),
        (2, 202, 'Seleção', 'Entrevista final', 'Carlos Pereira', '11965432109', 'Entrevista final marcada.', '2024-03-04'),
        (3, 301, 'Avaliação de Desempenho', 'Revisão do desempenho', 'Ana Souza', '11954321098', 'Coleta de feedbacks em andamento.', '2024-03-05'),
        (3, 302, 'Avaliação de Desempenho', 'Reunião de feedback', 'Carlos Pereira', '11965432109', 'Reunião agendada com gerente.', '2024-03-06'),
        (4, 401, 'Solicitação de Férias', 'Pedido formalizado', 'João Silva', '11987654321', 'Pedido de férias registrado.', '2024-03-07')
    ]
   
    LOGGER.info(f"Inserindo {len(dados_iniciais)} registros na tabela")
    cursor.executemany("""
        INSERT INTO processos_andamento
        (Codigo_Processo, Codigo_Atividade, Nome_Processo, Nome_Atividade, Nome_Cliente, Telefone_Cliente, Descricao_Processo, Data_Atividade)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?)
    """, dados_iniciais)
    
    # Verificar número de registros inseridos
    cursor.execute("SELECT COUNT(*) FROM processos_andamento")
    count = cursor.fetchone()[0]
    LOGGER.info(f"Total de registros na tabela: {count}")
    
    # Salvar e fechar a conexão
    conn.commit()
    conn.close()
    
    LOGGER.info(f"Banco de dados SQLite configurado com sucesso em {DB_PATH}")
    
except sqlite3.Error as e:
    LOGGER.error(f"Erro SQLite: {e}")
except Exception as e:
    LOGGER.error(f"Erro inesperado: {e}")

2025-03-25 13:19:26,110 - INFO - Iniciando configuração do banco de dados em ../.db/SQL_AGENT.db
2025-03-25 13:19:26,176 - INFO - Conexão com o banco de dados estabelecida com sucesso
2025-03-25 13:19:26,242 - INFO - Criando tabela 'processos_andamento'
2025-03-25 13:19:26,280 - INFO - Inserindo 7 registros na tabela
2025-03-25 13:19:26,288 - INFO - Total de registros na tabela: 7
2025-03-25 13:19:26,306 - INFO - Banco de dados SQLite configurado com sucesso em ../.db/SQL_AGENT.db


### 🗄️ Criando o Gerenciador de Banco de Dados  

Ótimo! 🎉 Com a base de dados pronta, é hora de criar a classe responsável por manipulá-la.  

Essa classe é relativamente simples, mas **essencial** para o projeto. Ela funciona como um **intermediário** entre os agentes e o banco de dados **SQLite**, permitindo que eles interajam com os dados de forma organizada. 🏗️🔗  

🔹 **Principais métodos:**  

📜 **`get_schema() -> str`**: Retorna o esquema das tabelas no banco 🏛️, o que será útil para os agentes entenderem a estrutura dos dados.  

⚡ **`execute_query(query: str) -> List[Any]`**: Executa uma **query SQL** e retorna os resultados. Se for uma consulta (`SELECT`), os dados são formatados como **dicionários** 📊 para facilitar a manipulação.  

🔒 **`close()`**: Fecha a conexão com o banco de dados, garantindo que os recursos sejam liberados corretamente. ✅

In [5]:
class DatabaseManager:
    def __init__(self):
        """Inicializa o gerenciador de banco de dados com SQLite."""
        self.db_path = DB_PATH
        LOGGER.info(f"Inicializando DatabaseManager com banco de dados em {self.db_path}")
        try:
            self.connection = sqlite3.connect(self.db_path)
            self.connection.row_factory = sqlite3.Row  # Permite acessar os resultados por nome de coluna
            LOGGER.info("Conexão com o banco de dados estabelecida com sucesso\n")
        except sqlite3.Error as e:
            LOGGER.error(f"Erro ao conectar ao banco de dados: {str(e)}\n")
            raise Exception(f"Falha na conexão com o banco de dados: {str(e)}")

    def get_schema(self) -> str:
        """Recupera o esquema do banco de dados SQLite."""
        LOGGER.info("Obtendo esquema do banco de dados")
        try:
            cursor = self.connection.cursor()
            cursor.execute("SELECT name, sql FROM sqlite_master WHERE type='table';")
            schema_info = cursor.fetchall()
            
            tables_count = len(schema_info)
            LOGGER.debug(f"Encontradas {tables_count} tabelas no banco de dados")
            
            schema = "\n".join(f"Table: {row['name']}\n{row['sql']}" for row in schema_info if row['sql'])
            LOGGER.debug(f"Esquema obtido: {schema}\n")
            return schema
        except sqlite3.DatabaseError as e:
            error_msg = f"Erro ao obter o esquema do banco de dados: {str(e)}\n"
            LOGGER.error(error_msg)
            raise Exception(error_msg)

    def execute_query(self, query: str) -> List[Dict[str, Any]]:
        """Executa uma query SQL no banco SQLite e retorna os resultados."""
       
        LOGGER.info(f"Executando query: {query}")
        
        try:
            is_select = query.strip().lower().startswith("select")
            
            if not is_select:
                operacao = query.strip().split("/")[0]
                LOGGER.warning(f"Tentativa de execução de operação SQL '{operacao}' não permitida")
                raise Exception(f"Operação SQL '{operacao}' não permitida")
            
            cursor = self.connection.cursor()
            start_time = datetime.now()
            cursor.execute(query)
            
            results = [dict(row) for row in cursor.fetchall()]           
            
            execution_time = (datetime.now() - start_time).total_seconds()
            LOGGER.info(f"Tempo de execução: {execution_time:.3f} segundos\n")

            return results
        except sqlite3.DatabaseError as e:
            error_msg = f"Erro ao executar a consulta: {str(e)}\n"
            LOGGER.error(error_msg)            
            raise Exception(error_msg)

    def close(self):
        """Fecha a conexão com o banco de dados."""
        LOGGER.info("Fechando conexão com o banco de dados")
        try:
            self.connection.close()
            LOGGER.info("Conexão com o banco de dados fechada com sucesso\n")
        except sqlite3.Error as e:
            LOGGER.error(f"Erro ao fechar a conexão com o banco de dados: {str(e)}\n")

In [6]:
db_manager = DatabaseManager()

LOGGER.info(db_manager.get_schema())
LOGGER.info(db_manager.execute_query("SELECT * FROM processos_andamento")[0])

2025-03-25 13:19:37,357 - INFO - Inicializando DatabaseManager com banco de dados em ../.db/SQL_AGENT.db
2025-03-25 13:19:37,371 - INFO - Conexão com o banco de dados estabelecida com sucesso

2025-03-25 13:19:37,372 - INFO - Obtendo esquema do banco de dados
2025-03-25 13:19:37,377 - INFO - Table: sqlite_sequence
CREATE TABLE sqlite_sequence(name,seq)
Table: processos_andamento
CREATE TABLE processos_andamento (
        ID_Processo_Andamento INTEGER PRIMARY KEY AUTOINCREMENT,
        Codigo_Processo INTEGER NOT NULL,
        Codigo_Atividade INTEGER NOT NULL,
        Nome_Processo TEXT NOT NULL,
        Nome_Atividade TEXT NOT NULL,
        Nome_Cliente TEXT NOT NULL,
        Telefone_Cliente TEXT NOT NULL,
        Descricao_Processo TEXT NOT NULL,
        Data_Atividade DATE NOT NULL
    )
2025-03-25 13:19:37,379 - INFO - Executando query: SELECT * FROM processos_andamento
2025-03-25 13:19:37,380 - INFO - Tempo de execução: 0.000 segundos

2025-03-25 13:19:37,382 - INFO - {'ID_Proces

### 🤖 Criando o Gerenciador de LLM  

O que dá inteligência aos agentes é o **LLM** (*Large Language Model*), então nada mais justo do que criar um gerenciador específico para ele.  

Neste exemplo, será utilizado o modelo **`deepseek-r1-distill-llama-70b`** via **GROQ**, pois essa opção possui uma camada *free* bem generosa. Além disso, esse modelo tem a capacidade de **refletir antes de gerar uma resposta** 🧠💡, o que pode influenciar positivamente nos resultados. Bem, veremos se isso realmente faz diferença! 🤔  

O **LLMManager** será responsável por interagir com o modelo e gerar respostas com base nos prompts fornecidos. Essa classe encapsula a comunicação com a API do modelo de **LLM**, permitindo uma interação simples e organizada.  

⚙️ **Principais métodos:**  

🔹 **`__init__()`**: Inicializa a conexão com a API da **GROQ**, configurando o modelo escolhido e os parâmetros principais:  
- 🎯 `temperature=0.1`: Mantém as respostas mais determinísticas, reduzindo a criatividade excessiva.  
- 🔄 `max_retries=2`: Define um limite de tentativas em caso de falha na requisição.  

🔹 **`invoke(prompt, **kwargs) -> str`**:  
- Recebe um 📝 **`ChatPromptTemplate`**, que contém o formato da mensagem.  
- Formata os dados necessários e envia a requisição para o **LLM**.  
- Retorna a resposta gerada pelo modelo. 🚀

In [7]:
class LLMManager:
    def __init__(self):
        """Inicializa o gerenciador de LLM com o modelo Groq."""
        LOGGER.info("Inicializando LLMManager com o modelo deepseek-r1-distill-llama-70b")
        
        api_key = os.getenv("GROQ_API_KEY")
        if not api_key:
            LOGGER.error("GROQ_API_KEY não encontrada nas variáveis de ambiente\n")
            raise ValueError("GROQ_API_KEY não configurada. Configure a variável de ambiente GROQ_API_KEY.")
        
        try:
            self.llm = ChatGroq(
                model="deepseek-r1-distill-llama-70b",
                api_key=api_key,
                temperature=0.1,
                max_retries=2,
            )
            LOGGER.info("LLM inicializado com sucesso (modelo: deepseek-r1-distill-llama-70b, temperatura: 0.1)\n")
        except Exception as e:
            LOGGER.error(f"Erro ao inicializar o LLM: {str(e)}\n")
            raise Exception(f"Falha na inicialização do LLM: {str(e)}")

    def invoke(self, prompt: ChatPromptTemplate, **kwargs) -> str:
        """
        Invoca o LLM com o prompt fornecido e parâmetros adicionais.
        
        Args:
            prompt: O template de prompt do chat
            **kwargs: Variáveis para formatação do prompt
            
        Returns:
            str: A resposta do modelo
        """       
        
        try:           
            start_format_time = time.time()
            messages = prompt.format_messages(**kwargs)
            format_time = time.time() - start_format_time
            
            LOGGER.info("Enviando requisição ao modelo...")
            start_invoke_time = time.time()
            response = self.llm.invoke(messages)
            invoke_time = time.time() - start_invoke_time
           
            response_content = response.content
            LOGGER.debug(f"Resposta recebida em {invoke_time:.3f}s ({len(response_content)} caracteres)")
            LOGGER.info(f"Resposta do LLM: {response_content}")
            
            # Log de métricas
            total_time = format_time + invoke_time
            LOGGER.debug(f"Invocação completa. Tempo total: {total_time:.3f}s")
            
            return re.sub(r'<think>.*?</think>\s*', '', response_content, flags=re.DOTALL)
            
        except Exception as e:
            LOGGER.error(f"Erro ao invocar o LLM: {str(e)}")
            raise Exception(f"Falha na invocação do LLM: {str(e)}")

In [8]:
llm_manager = LLMManager()

template = ChatPromptTemplate([
    ("system", "Seu nome é Brian, você está sempre feliz e alegre, sempre respondendo em PT-BR."),
    ("human", "Olá, meu nome é Rodrigo, e o seu?"),   
])

llm_manager.invoke(template)

2025-03-25 13:20:00,070 - INFO - Inicializando LLMManager com o modelo deepseek-r1-distill-llama-70b
2025-03-25 13:20:04,121 - INFO - LLM inicializado com sucesso (modelo: deepseek-r1-distill-llama-70b, temperatura: 0.1)

2025-03-25 13:20:04,171 - INFO - Enviando requisição ao modelo...
2025-03-25 13:20:05,947 - INFO - Resposta do LLM: <think>
Okay, so I just received a message from Rodrigo. He said, "Olá, meu nome é Rodrigo, e o seu?" which means "Hello, my name is Rodrigo, and yours?" in Portuguese. I need to respond appropriately.

First, I should greet him back. Since he used "Olá," I can respond with the same or maybe a slightly more enthusiastic greeting like "Olá, Rodrigo!" to make it friendly.

Next, I need to introduce myself. My name is Brian, so I'll include that. Since I'm supposed to always be happy and cheerful, I should add an emoji to convey that emotion. Maybe a smiling face or something similar.

I should also ask him how he's doing to keep the conversation going. So,

'Olá, Rodrigo! Eu sou o Brian! 😊 Como você está?'

### 🤖 Analisando a Pergunta do Usuário  

Ao lidar com um problema envolvendo 🗄️ **banco de dados**, a primeira tarefa é identificar 🔎 **quais tabelas e colunas** fazem parte da solução. Neste caso, existe apenas **uma tabela**, pois toda a lógica para populá-la será resolvida sem a necessidade de um sistema de agentes para isso.  

Com essa base estabelecida, é possível testar se o 🏗️ **Agente** criado, utilizando o 🤖 **LLM**, consegue **abstrair a lógica necessária** para interpretar uma pergunta e retornar as 📊 **colunas relevantes**.  

### 🚀 Técnicas Essenciais de Prompt Engineering  

Ao usar **técnicas de engenharia de prompt (Prompt Engineering)**, é possível melhorar a capacidade do LLM de entender e gerar consultas corretas.  

Para criar um agente eficiente, deve-se utilizar algumas estratégias fundamentais:  

🔹 **Definição de Persona**: Faz com que o modelo assuma o papel de um **analista de dados**, influenciando seu estilo de resposta.  
🔹 **Instrução Clara e Específica**: Detalha exatamente o que é esperado do modelo, reduzindo ambiguidades.  
🔹 **Formato de Saída Especificado**: Garante que o modelo retorne um **JSON estruturado**, facilitando o processamento da resposta.  
🔹 **Restrições e Regras Detalhadas**: Define os limites para que o modelo foque nas colunas **relevantes**, ignorando campos irrelevantes.  
🔹 **Injeção de Contexto**: É passado o **esquema do banco de dados** para que o modelo compreenda melhor a estrutura disponível.  
🔹 **Uso de Delimitadores**: Organiza a entrada do modelo, separando **esquema do banco** e **pergunta do usuário**, melhorando a compreensão.

💡 Essa abordagem permite que o modelo compreenda a **estrutura do banco** e selecione apenas as colunas relevantes com mais eficiência! 🚀

In [9]:
def analisar_pergunta(pergunta: str) -> Dict[str, Any]:
    """
    Analisa a pergunta do usuário e identifica as tabelas e colunas relevantes.
    
    Args:
        pergunta (str): Pergunta do usuário a ser analisada
        
    Returns:
        Dict[str, Any]: Dicionário contendo a análise da pergunta
    """    

    LOGGER.info(f"Iniciando análise da pergunta: '{pergunta}'")
    
    try:        
        start_schema_time = time.time()
        esquema = db_manager.get_schema()
        
        LOGGER.debug(f"Preparando prompt para o LLM")
        prompt = ChatPromptTemplate.from_messages([
            ("system", '''Você é um analista de dados que pode ajudar a resumir tabelas SQL e interpretar perguntas de usuários sobre um banco de dados.  
Dada a pergunta e o esquema do banco de dados, identifique as tabelas e colunas relevantes.  
Se a pergunta não for relevante para o banco de dados ou se não houver informações suficientes para respondê-la, defina "is_relevante" como false.
Sua resposta deve estar no seguinte formato JSON:
{{
    "is_relevante": boolean,
    "tabelas_relevantes": [
        {{
            "nome_tabela": string,
            "colunas": [string],
            "colunas_substantivo": [string]
        }}
    ]
}}
O campo "colunas_substantivo" deve conter apenas as colunas que são relevantes para a pergunta e que contêm substantivos ou nomes.  
Por exemplo, a coluna "Nome do Artista" contém substantivos relevantes para a pergunta "Quais são os artistas mais vendidos?",  
mas a coluna "ID do Artista" não é relevante, pois não contém um substantivo. Não inclua colunas que contenham números.
'''),
            ("human", "===Esquema do banco de dados:\n{esquema}\n\n===Pergunta do usuário:\n{pergunta}\n\nIdentifique as tabelas e colunas relevantes:")
        ])        
       
        analisador_json = JsonOutputParser()
         
        resposta = llm_manager.invoke(prompt, esquema=esquema, pergunta=pergunta)          

        LOGGER.debug(f"Analisando resposta JSON")      
        try:
            resposta_analisada = analisador_json.parse(resposta) 
            
            is_relevante = resposta_analisada.get("is_relevante", False)
            num_tabelas = len(resposta_analisada.get("tabelas_relevantes", []))
            LOGGER.info(f"Análise concluída. Relevante: {is_relevante}, Tabelas identificadas: {num_tabelas}")
            
            if num_tabelas > 0:
                tabelas_nomes = [tabela.get("nome_tabela") for tabela in resposta_analisada.get("tabelas_relevantes", [])]
                LOGGER.debug(f"Tabelas relevantes: {', '.join(tabelas_nomes)}")
            
        except Exception as e:
            LOGGER.error(f"Erro ao analisar JSON da resposta: {str(e)}")
            LOGGER.error(f"Resposta que causou o erro: {resposta}\n")
            # Se houver erro no parsing, retornamos uma resposta de fallback
            resposta_analisada = {"is_relevante": False, "tabelas_relevantes": []}
        
        LOGGER.info(f"Resultado Final da Análise: {str(resposta_analisada)}")
        
        # Tempo total da operação
        total_time = time.time() - start_schema_time
        LOGGER.info(f"Análise completa em {total_time:.3f}s")
        
        # Retornar o resultado
        return {"pergunta_analisada": resposta_analisada}
        
    except Exception as e:
        LOGGER.error(f"Erro durante análise da pergunta: {str(e)}\n")
        
        return {"pergunta_analisada": {"is_relevante": False, "tabelas_relevantes": []}}

In [10]:
pergunta = "Do que se trada o processo da Ana Souza?"
pergunta_analisada = analisar_pergunta(pergunta)['pergunta_analisada']

2025-03-25 13:20:31,612 - INFO - Iniciando análise da pergunta: 'Do que se trada o processo da Ana Souza?'
2025-03-25 13:20:31,614 - INFO - Obtendo esquema do banco de dados
2025-03-25 13:20:31,617 - INFO - Enviando requisição ao modelo...
2025-03-25 13:20:35,016 - INFO - Resposta do LLM: <think>
Ok, vamos analisar a pergunta do usuário: "Do que se trada o processo da Ana Souza?" Primeiro, preciso entender o que o usuário está procurando. Parece que ele quer saber qual é o processo relacionado a uma pessoa específica, Ana Souza.

Agora, olhando para o esquema do banco de dados, temos duas tabelas: sqlite_sequence e processos_andamento. A tabela sqlite_sequence parece ser do sistema e não contém informações relevantes para a pergunta, então podemos ignorá-la.

A tabela processos_andamento tem várias colunas: ID_Processo_Andamento, Codigo_Processo, Codigo_Atividade, Nome_Processo, Nome_Atividade, Nome_Cliente, Telefone_Cliente, Descricao_Processo e Data_Atividade. 

A pergunta menciona "

### 🔍 Encontrando Substantivos Únicos em Tabelas e Colunas Relevantes  

Ao utilizar um 🤖 **LLM** para processar consultas, é essencial ✅ **validar a resposta gerada**. Essa etapa do processo tem como objetivo 🛠️ **higienizar o retorno**, garantindo que cada 📊 **coluna** seja referenciada apenas uma vez.

In [ ]:
def obter_substantivos_unicos(pergunta_analisada: dict) -> dict:
    """Identifica substantivos únicos nas tabelas e colunas relevantes."""
    
    if not pergunta_analisada['is_relevante']:
        LOGGER.info("A pergunta não é relevante. Retornando lista vazia.\n")
        return {"substantivos_unicos": []}

    substantivos_unicos = set()
    LOGGER.info("Iniciando a busca por substantivos únicos.")

    for info_tabela in pergunta_analisada['tabelas_relevantes']:
        nome_tabela = info_tabela['nome_tabela']
        colunas_substantivos = info_tabela['colunas_substantivo']

        if not colunas_substantivos:
            LOGGER.debug(f"A tabela '{nome_tabela}' não possui colunas relevantes.")
            continue

        nomes_colunas = ', '.join(f"`{col}`" for col in colunas_substantivos)
        consulta = f"SELECT DISTINCT {nomes_colunas} FROM `{nome_tabela}`"
        LOGGER.debug(f"Executando consulta SQL na tabela '{nome_tabela}': {consulta}")

        try:
            resultados = db_manager.execute_query(consulta)
            LOGGER.debug(f"Consulta retornou {len(resultados)} registros.")

            for linha in resultados:
                valores = [str(valor) for valor in linha if valor]
                substantivos_unicos.update(valores)
                LOGGER.debug(f"Valores extraídos: {valores}")

        except Exception as e:
            LOGGER.error(f"Erro ao executar consulta na tabela '{nome_tabela}': {e}\n")

    LOGGER.info(f"Processo concluído. {len(substantivos_unicos)} substantivos únicos encontrados.")
    LOGGER.info(f"Substantivos Únicos: {str(substantivos_unicos)}\n")
    return {"substantivos_unicos": list(substantivos_unicos)}


In [ ]:
substantivos_unicos = obter_substantivos_unicos(pergunta_analisada)['substantivos_unicos']

2025-03-25 13:21:16,453 - INFO - Iniciando a busca por substantivos únicos.
2025-03-25 13:21:16,455 - INFO - Executando query: SELECT DISTINCT `Nome_Cliente`, `Descricao_Processo` FROM `processos_andamento`
2025-03-25 13:21:16,459 - INFO - Tempo de execução: 0.003 segundos

2025-03-25 13:21:16,459 - INFO - Processo concluído. 2 substantivos únicos encontrados.
2025-03-25 13:21:16,459 - INFO - Substantivos Únicos: {'Nome_Cliente', 'Descricao_Processo'}



### ✨ Gerando uma consulta SQL com base na pergunta analisada e nos substantivos únicos  

Sabe aquele momento em que o filme chega ao seu ápice? 🎬 Pois é, estamos exatamente aí! Já temos a pergunta do usuário, uma análise das colunas relevantes e as colunas devidamente higienizadas. Ou seja, temos insumos suficientes para criar um agente que irá gerar a consulta SQL. *Maravilhindo!* 🚀  

Aproveitando o embalo, vamos falar sobre mais uma técnica de prompt:  

- **Few-shot Prompting (Exemplos)**: Essa técnica consiste em fornecer ao modelo alguns exemplos de entrada e suas respectivas saídas antes da pergunta principal. Isso ajuda o modelo a entender o formato esperado da resposta e a aprender o padrão da tarefa, melhorando a qualidade da geração. 🎯  

In [13]:
def gerar_sql(pergunta: str, pergunta_analisada: dict, substantivos_unicos: list) -> dict:
    """Gera uma consulta SQL com base na pergunta analisada e nos substantivos únicos."""
    
    LOGGER.info(f"Iniciando geração da consulta SQL para a pergunta: '{pergunta}'")
    start_time = time.time()

    if not pergunta_analisada['is_relevante']:
        LOGGER.warning("Pergunta não é relevante.")
        return {"consulta_sql": "NAO_RELEVANTE", "is_relevante": False}
    
    try:
        esquema = db_manager.get_schema()
        
        LOGGER.debug("Preparando prompt para o LLM")
        prompt = ChatPromptTemplate.from_messages([
            ("system", '''
    Você é um assistente de IA que gera consultas SQL com base na pergunta do usuário, no esquema do banco de dados e nos substantivos únicos encontrados nas tabelas relevantes. Gere uma consulta SQL válida para responder à pergunta do usuário.

    Se não houver informações suficientes para escrever uma consulta SQL, responda com "INFORMACAO_INSUFICIENTE".

    Aqui estão alguns exemplos:

    1. Qual é o produto mais vendido?
    Resposta: SELECT product_name, SUM(quantity) as total_quantity FROM sales WHERE product_name IS NOT NULL AND quantity IS NOT NULL AND product_name != "" AND quantity != "" AND product_name != "N/A" AND quantity != "N/A" GROUP BY product_name ORDER BY total_quantity DESC LIMIT 1

    2. Qual é a receita total para cada produto?
    Resposta: SELECT \`product name\`, SUM(quantity * price) as total_revenue FROM sales WHERE \`product name\` IS NOT NULL AND quantity IS NOT NULL AND price IS NOT NULL AND \`product name\` != "" AND quantity != "" AND price != "" AND \`product name\` != "N/A" AND quantity != "N/A" AND price != "N/A" GROUP BY \`product name\`  ORDER BY total_revenue DESC

    3. Qual é a participação de mercado de cada produto?
    Resposta: SELECT \`product name\`, SUM(quantity) * 100.0 / (SELECT SUM(quantity) FROM sales) as market_share FROM sales WHERE \`product name\` IS NOT NULL AND quantity IS NOT NULL AND \`product name\` != "" AND quantity != "" AND \`product name\` != "N/A" AND quantity != "N/A" GROUP BY \`product name\`  ORDER BY market_share DESC
   
    Apenas forneça a string da consulta SQL. Não a formate. Certifique-se de usar a grafia correta dos substantivos conforme fornecido na lista de substantivos únicos. Todos os nomes de tabelas e colunas devem estar entre crases.
    '''),
            ("human", '''===Esquema do banco de dados:
    {esquema}

    ===Pergunta do usuário:
    {pergunta}

    ===Tabelas e colunas relevantes:
    {pergunta_analisada}

    ===Substantivos únicos nas tabelas relevantes:
    {substantivos_unicos}

    Gere a string da consulta SQL'''),
        ])
       
        consulta = llm_manager.invoke(
            prompt, 
            esquema=esquema, 
            pergunta=pergunta, 
            pergunta_analisada=pergunta_analisada, 
            substantivos_unicos=substantivos_unicos
        )

        if consulta.strip() == "INFORMACAO_INSUFICIENTE":
            return {"consulta_sql": "NAO_RELEVANTE"}
        
        total_time = time.time() - start_time
        LOGGER.info(f"Consulta SQL gerada com sucesso em {total_time:.3f}s")
        LOGGER.info(f"Consulta SQL: {consulta}\n")
        
        return {"consulta_sql": consulta}
        
    except Exception as e:
        LOGGER.error(f"Erro durante a geração da consulta SQL: {str(e)}\n")
        return {"consulta_sql": "ERRO_EXECUCAO", "is_relevante": False}



In [14]:
consulta_sql = gerar_sql(pergunta, pergunta_analisada, substantivos_unicos)['consulta_sql']

2025-03-25 13:21:28,820 - INFO - Iniciando geração da consulta SQL para a pergunta: 'Do que se trada o processo da Ana Souza?'
2025-03-25 13:21:28,821 - INFO - Obtendo esquema do banco de dados
2025-03-25 13:21:28,830 - INFO - Enviando requisição ao modelo...
2025-03-25 13:21:33,077 - INFO - Resposta do LLM: <think>
Okay, I need to help the user by generating a SQL query based on their question. The question is "Do que se trata o processo da Ana Souza?" which translates to "What is Ana Souza's process about?" 

First, I look at the database schema provided. There's a table called processos_andamento with several columns. The relevant columns for this question are Nome_Cliente (Client's Name) and Descricao_Processo (Process Description). 

The user wants to know the description of the process for a specific client, Ana Souza. So, I need to select the Descricao_Processo from the processos_andamento table where Nome_Cliente is 'Ana Souza'.

I should make sure to use the correct column nam

## ✅ Validando e Corrigindo a Consulta SQL Gerada  

Se sua memória for boa, você deve se lembrar que validar e corrigir a resposta de um LLM é uma prática recomendada. Essa abordagem tem sido amplamente adotada pela comunidade, garantindo maior precisão e confiabilidade nas respostas.  

Portanto, antes de utilizarmos a consulta SQL gerada, vamos validar se o outro agente trabalhou como esperado.  

As técnicas utilizadas aqui não são nenhuma novidade, mas são essenciais para evitar erros e garantir um resultado mais confiável. ⚡  


Agora, com esse processo de validação e correção, garantimos que a consulta SQL gerada esteja correta antes de ser executada. 🔍💡

In [15]:
def validar_e_corrigir_sql(consulta_sql) -> dict:
    """Valida e corrige a consulta SQL gerada."""
           
    LOGGER.info(f"Iniciando validação da consulta SQL: '{consulta_sql}'")
    start_time = time.time()
    
    if consulta_sql == "NAO_RELEVANTE":
        LOGGER.warning("A consulta não é relevante.")
        return {"consulta_sql": "NAO_RELEVANTE", "consulta_valida": False}
    
    try:
        
        esquema = db_manager.get_schema()

        prompt = ChatPromptTemplate.from_messages([
            ("system", '''
    Você é um assistente de IA que valida e corrige consultas SQL. Sua tarefa é:
    1. Verificar se a consulta SQL é válida.
    2. Garantir que todos os nomes de tabelas e colunas estejam corretamente escritos e existam no esquema do banco de dados. Todos os nomes de tabelas e colunas devem estar entre crases.
    3. Se houver problemas, corrija-os e forneça a consulta SQL corrigida.
    4. Se não houver problemas, retorne a consulta original.

   Responda no formato JSON com a seguinte estrutura. Responda apenas com o JSON:
    {{
        "valido": booleano,
        "problemas": string ou null,
        "consulta_corrigida": string
    }}

    Por exemplo:
    1. 
    {{
        "valido": true,
        "problemas": null,
        "consulta_corrigida": "None"
    }}
                
    2. 
    {{
        "valido": false,
        "problemas": "A coluna USERS não existe",
        "consulta_corrigida": "SELECT * FROM users WHERE age > 25"
    }}

    3. 
    {{
        "valido": false,
        "problemas": "Os nomes de colunas e tabelas devem estar entre crases se contiverem espaços",
        "consulta_corrigida": "SELECT * FROM \`gross income\` WHERE \`age\` > 25"
    }}
    '''),
            ("human", '''===Esquema do banco de dados:
    {esquema}

    ===Consulta SQL gerada:
    {consulta_sql}
                
    '''),
        ])

        analisador_saida = JsonOutputParser()
        
        resposta = llm_manager.invoke(prompt, esquema=esquema, consulta_sql=consulta_sql)
        resultado = analisador_saida.parse(resposta)

        if resultado["valido"] and resultado["problemas"] is None:
            total_time = time.time() - start_time
            LOGGER.info(f"Consulta SQL validade em {total_time:.3f}s")
            LOGGER.info(f"Consulta SQL: {resultado['consulta_corrigida']}\n")
            
            return {"consulta_corrigida": consulta_sql, "consulta_valida": True, "consulta_problemas": None}
        
        total_time = time.time() - start_time
        LOGGER.info(f"Consulta SQL validade em {total_time:.3f}s")
        LOGGER.info(f"Consulta SQL: {resultado['consulta_corrigida']}\n")
        
        
        return {
            "consulta_corrigida": resultado["consulta_corrigida"],
            "consulta_valida": resultado["valido"],
            "consulta_problemas": resultado["problemas"]
        }
    except Exception as e:
        LOGGER.error(f"Erro ao validar a consulta SQL: {e}\n", exc_info=True)
        return {"consulta_sql": "ERRO_EXECUCAO", "consulta_valida": False, "consulta_problemas": str(e)}


In [16]:
consulta_corrigida = validar_e_corrigir_sql(consulta_sql)
LOGGER.info(consulta_corrigida)

2025-03-25 13:21:57,670 - INFO - Iniciando validação da consulta SQL: 'SELECT `Descricao_Processo` FROM `processos_andamento` WHERE `Nome_Cliente` = 'Ana Souza''
2025-03-25 13:21:57,672 - INFO - Obtendo esquema do banco de dados
2025-03-25 13:21:57,675 - INFO - Enviando requisição ao modelo...
2025-03-25 13:22:00,261 - INFO - Resposta do LLM: <think>
Okay, I need to validate and correct the given SQL query based on the provided database schema. Let me start by understanding the task.

First, I'll check if the SQL query is valid. The query is a SELECT statement, which is a standard SQL command, so that's good. It selects the `Descricao_Processo` column from the `processos_andamento` table where `Nome_Cliente` is 'Ana Souza'. 

Next, I need to ensure that all table and column names are correctly written and exist in the schema. The table name is `processos_andamento`, which matches the schema. Now, checking the columns: `Descricao_Processo` and `Nome_Cliente` both exist in the table as p

### 🚀 Executando a consulta SQL gerada 🔍
Agora saberemos se o objetivo foi alcançado ✅. Nada de LLM, somente a boa e velha QUERY ⚡💻.

In [17]:
def executar_sql(consulta: str) -> dict:
    """Executa a consulta SQL e retorna os resultados."""
    
    LOGGER.info(f"Iniciando a execução da consulta SQL: '{consulta_sql}'")
    start_time = time.time()
    
    if consulta == "NAO_RELEVANTE":
        LOGGER.warning("A consulta não é relevante.")
        return {"resultados": "NAO_RELEVANTE"}

    try:
        resultados = db_manager.execute_query(consulta)
        
        total_time = time.time() - start_time
        LOGGER.info(f"Consulta SQL exedcutada em {total_time:.3f}s")
        LOGGER.info(f"Resultado: {str(resultados)}\n")
        
        return {"resultados": resultados}
    except Exception as e:
        LOGGER.error(f"Erro ao executar a consulta SQL: {e}\n", exc_info=True)
        return {"erro": str(e)}

In [20]:
resultados = executar_sql(consulta_corrigida['consulta_corrigida']) 
colunas = resultados["resultados"][0].keys()

print(f"Solicitação: {pergunta}\n")

print(" | ".join(colunas))
print("-" * (len(" | ".join(colunas)) + 5))

for linha in resultados["resultados"]:
    print(" | ".join(str(valor) for valor in linha.values()))

2025-03-25 13:23:39,196 - INFO - Iniciando a execução da consulta SQL: 'SELECT `Descricao_Processo` FROM `processos_andamento` WHERE `Nome_Cliente` = 'Ana Souza''
2025-03-25 13:23:39,199 - INFO - Executando query: SELECT `Descricao_Processo` FROM `processos_andamento` WHERE `Nome_Cliente` = 'Ana Souza'
2025-03-25 13:23:39,201 - INFO - Tempo de execução: 0.001 segundos

2025-03-25 13:23:39,202 - INFO - Consulta SQL exedcutada em 0.004s
2025-03-25 13:23:39,203 - INFO - Resultado: [{'Descricao_Processo': 'Coleta de feedbacks em andamento.'}]



Solicitação: Do que se trada o processo da Ana Souza?

Descricao_Processo
-----------------------
Coleta de feedbacks em andamento.


### 🏆 Conclusão  

Os resultados alcançados demonstram a eficácia e sofisticação do processo desenvolvido. 🚀 A abordagem adotada conseguiu abstrair com precisão a lógica de criação de consultas SQL, permitindo que o LLM executasse sua tarefa com alta precisão e consistência. O fluxo foi projetado para minimizar erros e maximizar a qualidade das respostas geradas, garantindo que cada consulta fosse validada e corrigida de forma inteligente.  

Este projeto não se trata apenas de gerar SQLs — ele representa uma integração avançada entre inteligência artificial e engenharia de software, combinando automação, validação e refinamento contínuo. 💡 Quanto mais os agentes forem aprimorados, mais refinadas e confiáveis se tornarão as respostas, elevando o nível da automação na geração e validação de consultas.  

O caminho até aqui mostrou que é possível criar sistemas que combinam o poder dos LLMs com processos rigorosos de controle e melhoria contínua. E isso é apenas o começo. 😉🔥

### 🚀 Próximos Passos  

Um ponto final é apenas o começo de uma nova história. 📖✨ Seguindo essa lógica, este projeto continuará evoluindo para se tornar ainda mais sofisticado e eficiente. Algumas das próximas funcionalidades que serão implementadas incluem:  

- 🔹 Criar um agente capaz de gerar respostas diretas para o usuário.  
- 🔹 Construir um grafo com **LangGraph** para estruturar melhor o fluxo das interações.  
- 🔹 Desenvolver um bot com **Chainlit** como interface interativa.  
- 🔹 Explorar a possibilidade de utilizar o **WhatsApp** como meio de comunicação.  
- 🔹 Adicionar um campo na base de dados que registre o histórico do processo, permitindo que o bot use essas informações para interagir de forma mais contextualizada.  
- 🔹 Implementar uma memória de contexto, garantindo que o bot se lembre do que foi discutido ao longo da conversa.  
- 🔹 Criar uma memória de longo prazo para armazenar consultas bem-sucedidas e melhorar a geração de novas consultas SQL.  
- 🔹 Desenvolver um mecanismo de correção automática, onde erros detectados sejam enviados ao agente responsável para ajuste antes de seguir adiante.  


### 🔍 Possíveis Melhorias Futuras  

Além das funcionalidades planejadas, algumas ideias adicionais podem agregar ainda mais valor ao projeto:  

1️⃣ **Refinamento contínuo do agente de validação**: Melhorar os critérios de correção do SQL e integrar um modelo especializado em análise de consultas.  

2️⃣ **Monitoramento e análise de uso**: Criar dashboards que permitam visualizar o desempenho do sistema, identificando padrões de erro e áreas de melhoria.

3️⃣  **Personalização por usuário**: Permitir que o sistema aprenda com cada usuário, ajustando respostas e sugestões com base em interações anteriores.

### 🛠️ Minhas Limitações  

Ainda tenho muitas dúvidas sobre o processo de criar consultas a partir de texto natural. O **TTQ (Text to Query)** ainda me parece um pouco complexo, mas acredito que isso irá melhorar com o tempo.  

- ❓ Como lidar com várias requisições vindas pelo **WhatsApp**?  
- 🔄 Como manter o **contexto** em uma conversa, considerando que o usuário pode fazer uma pergunta e, depois, outra relacionada?  
- 🤯 Como criar **boa parte daquilo que quero colocar no projeto**? Porque ter ideias incríveis é fácil, o difícil é fazê-las funcionar sem quebrar tudo no caminho! 😅

### 🚀 Como Imagino Parte da Implementação  

Já parti do princípio de que existe uma tabela com todas as informações necessárias. Por óbvio, ela deverá ser criada, e as informações precisam ser carregadas nela de alguma forma. A lógica que usaria seria a seguinte:  

- 🏗️ **Criar a tabela** que irá receber os processos.  
- 🔄 **Cada processo criado** dentro da ferramenta será responsável por registrar seu progresso nessa tabela, podendo ser por meio de uma integração de passagem de etapa.  
  - 📌 **Ao abrir o processo**, deve-se registrar o código dele nessa tabela.  
  - 🔁 **Ao passar por cada atividade**, ou pelo menos as mais relevantes para consulta, buscar pela referência do processo na tabela e atualizar as informações.  
- 🗂️ Como algumas informações podem ser específicas de um processo e não caberem em uma **coluna genérica**, pode-se criar uma coluna que contenha um **JSON** com detalhes específicos. Esses dados podem servir como **insumos para o Agente** que criará as respostas.  
- ⚡ Para lidar com **múltiplas requisições**, pode ser interessante utilizar uma **fila**, como **Kafka**. 🎯